<a href="https://colab.research.google.com/github/pascalghanimi/Injury-Prediction-in-Runners/blob/main/Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.serialization
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

data = []
labels = []
fahrstil_labels = []

MAX_LENGTH = 400
FEATURE_FILE = "PHALP_AAaron_1_features.pkl"
SCALER_FILE  = "scaler_schwung.pkl"
ENCODER_FILE = "encoder_schwung.pkl"
MODEL = "ski_schwung_classifier.pt"
INPUT_VIDEO     = "PHALP_AMichi_2.mp4"
OUTPUT_VIDEO    = "annotated_output.mp4"

# Scaler & Encoder laden
with open(SCALER_FILE, "rb") as f:
    scaler = pickle.load(f)
with open(ENCODER_FILE, "rb") as f:
    le = pickle.load(f)

# Features laden
with open(FEATURE_FILE, "rb") as f:
    features = pickle.load(f)

print(features.keys())


data = []

# printet alle Features
for feature in features:
  print(feature)

for feature in features:
  print(features[feature])



for frame in features["COM_to_ground"]:
    # geht über jeden Frame
    frame_features = [
        features["COM_to_ground"][frame],
        features["knee_angles_right"][frame],
        features["knee_angles_left"][frame]
    ]

    # Füge joint_angles hinzu
    for axis in features["joint_angles"][frame]:
        frame_features.extend(
            features["joint_angles"][frame][axis].values()
        )

    # Füge axis_angles hinzu
    for axis in features["axis_angles"][frame]:
        frame_features.extend(
            features["axis_angles"][frame][axis].values()
        )

    # Füge COM_angles einmalig hinzu (nicht in der axis-Schleife!)
    frame_features.extend(
        features["COM_angles"][frame].values()
    )

    # fügt für jeden Frame den kompletten Feature-Vektor hinzu
    data.append(frame_features)


data = np.array(data, dtype=np.float32)  # [n_frames, feat_dim]

# auf MAX_LENGTH padden oder trimmen
if data.shape[0] < MAX_LENGTH:
    pad = np.zeros((MAX_LENGTH - data.shape[0], data.shape[1]), dtype=np.float32)
    data = np.vstack([data, pad])
else:
    data = data[:MAX_LENGTH]


print("Data nach Numpy Array", data)


print("Data", data)


print("Length of Data", len(data)) #
print("Anzahl der Feature", len(data[0])) # Anzahl der Frames

data_scaled = scaler.transform(data)

x = torch.tensor(data_scaled, dtype=torch.float32).unsqueeze(0)


class SkiSwingLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.3
        )
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        _, (h, _) = self.lstm(x)
        h_cat = torch.cat((h[-2], h[-1]), dim=1)
        return self.fc(self.dropout(h_cat))

torch.serialization.add_safe_globals([SkiSwingLSTM, nn.LSTM])

model = torch.load(MODEL, map_location="cpu", weights_only=False)
model.eval()

with torch.no_grad():
    logits    = model(x)
    style_idx = logits.argmax(dim=1).item()             # <— hier style_idx setzen

style_str = le.inverse_transform([style_idx])[0]        # und hier style_str
print("Vorhergesagte Schwungart:", style_str)



dict_keys(['COM_to_ground', 'knee_angles_right', 'knee_angles_left', 'joint_angles', 'axis_angles', 'COM_angles', 'schwung_labels', 'fahrstil'])
COM_to_ground
knee_angles_right
knee_angles_left
joint_angles
axis_angles
COM_angles
schwung_labels
fahrstil
{0: np.float32(0.8962757), 1: np.float32(0.8856799), 2: np.float32(0.8710589), 3: np.float32(0.86827004), 4: np.float32(0.86714715), 5: np.float32(0.8514548), 6: np.float32(0.85463184), 7: np.float32(0.85151225), 8: np.float32(0.8469641), 9: np.float32(0.85062814), 10: np.float32(0.8470334), 11: np.float32(0.8490559), 12: np.float32(0.8443626), 13: np.float32(0.8438373), 14: np.float32(0.84143096), 15: np.float32(0.84114695), 16: np.float32(0.8358796), 17: np.float32(0.83632445), 18: np.float32(0.83740187), 19: np.float32(0.8394605), 20: np.float32(0.84096766), 21: np.float32(0.833881), 22: np.float32(0.82460475), 23: np.float32(0.82680744), 24: np.float32(0.8311478), 25: np.float32(0.8194066), 26: np.float32(0.8059989), 27: np.float32(

In [6]:
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.serialization

# ─── Einstellungen ────────────────────────────────────────────────────────────
# hier style_idx und style_str aus dem vorherigen Cell verfügbar

# ─── 1) Daten & Modelle laden ─────────────────────────────────────────────────
with open(FEATURE_FILE, "rb") as f:
    features = pickle.load(f)

class FrameWiseLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size,
                            num_layers=2, bidirectional=True,
                            batch_first=True, dropout=0.3)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(self.dropout(out))

torch.serialization.add_safe_globals([FrameWiseLSTM, nn.LSTM])
model_turn = torch.load("left_right_classifier.pt", map_location="cpu", weights_only=False)
model_turn.eval()

# ─── 2) Turn-Sequenz aufbauen (Style als letztes Feature) ───────────────────
frame_indices = sorted(features["COM_to_ground"].keys())
seq_turn = []
for i in frame_indices:
    try:
        v = [
            features["COM_to_ground"][i],
            features["knee_angles_right"][i],
            features["knee_angles_left"][i]
        ]
        for axis in features["joint_angles"][i]:
            v.extend(features["joint_angles"][i][axis].values())
        for axis in features["axis_angles"][i]:
            v.extend(features["axis_angles"][i][axis].values())
        v.extend(features["COM_angles"][i].values())
        v.append(style_idx)
        seq_turn.append(v)
    except KeyError:
        continue

if not seq_turn:
    raise RuntimeError("Turn-Sequence ist leer – bitte Features prüfen!")

arr_turn = np.array(seq_turn, dtype=np.float32)      # [n_frames, feat_dim+1]
x_turn   = torch.tensor(arr_turn).unsqueeze(0)       # [1, n_frames, feat_dim+1]

# ─── 3) Links/Rechts pro Frame vorhersagen ───────────────────────────────────
with torch.no_grad():
    preds = model_turn(x_turn).argmax(dim=-1).squeeze().numpy()

labels_turn = ["Linksschwung" if p == 1 else "Rechtsschwung" for p in preds]
print("Schwung pro Frame:", labels_turn)

# ─── 4) Video annotieren & speichern ─────────────────────────────────────────
import cv2
cap    = cv2.VideoCapture(INPUT_VIDEO)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps    = cap.get(cv2.CAP_PROP_FPS)
w      = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h      = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out    = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (w, h))

for idx, swing_txt in enumerate(labels_turn):
    ret, frame = cap.read()
    if not ret:
        break
    cv2.putText(frame, swing_txt,        (50, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
    cv2.putText(frame, f"Fahrstil: {style_str}", (50, h-30),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 2)
    out.write(frame)

cap.release()
out.release()
print(f"Annotated video saved as '{OUTPUT_VIDEO}'")


Schwung pro Frame: ['Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Rechtsschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Linksschwung', 'Lin